In [1]:
import pandas as pd
import ipywidgets as widget
from ipywidgets import Layout
from IPython.display import HTML, display, clear_output

import sys
sys.path.append('./core')
import data_report
import data_processing
import data_graphs

with open('template/variables.css') as f:
    css = f.read().replace(';', ' !important;')
display(HTML('<style type="text/css">%s</style>'%css))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
               
button_style = {'button_color' : '#5e6b74',
               'font_weight' : 'bold'}
items_layout = Layout(justify_content='center',
                       align_items='center',
                       width='150px')
box_layout = Layout(display='flex',
                    flex_flow='row',                    
                    justify_content='center',
                    align_items='center')
box_layout2 = Layout(display='flex',
                    flex_flow='column',                    
                    justify_content='center',
                    align_items='flex-start')

style_dataframe = [dict(selector="caption", props=[("color", "#5e6b74"), 
                                                   ("border-radius", "5px"), 
                                                   ("background", "#e7edf0"), 
                                                   ("font-weight", "bold"),
                                                   ("font-size", "20px")]), 
                   dict(selector="th.col_heading", props=[("color", "white"),
                                                          ("border-radius", "3px"),
                                                          ("background", "#5e6b74")]),
                   dict(selector="th.blank.level0", props=[("color", "#5e6b74"),
                                                          ("border-radius", "3px"),
                                                          ("background", "#5e6b74")]),
                   dict(selector="th", props=[("color", "black"),
                                              ("text-align", "center")]),
                   dict(selector="td", props=[("color", "black"),
                                              ("font-weight", "bold"),
                                              ("text-align", "center")])]
                        
year_text = 'Año'
year_label = widget.HTML(value = f"<b style='color:white; font-family:tahoma; font-size:18px; border-radius: 3px 3px 3px 3px'>{year_text}</b>")
year_menu = widget.Dropdown(options = [2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030],
                            value = 2021,                            
                            layout = items_layout)

month_text = 'Mes'
month_label = widget.HTML(value = f"<b style='color:white; font-family:tahoma; font-size:18px; border-radius: 3px 3px 3px 3px'>{month_text}</b>")
month_menu = widget.Dropdown(options = ['Enero', 'Febrero', 'Marzo', 'Abril', 
                                        'Mayo', 'Junio', 'Julio', 'Agosto', 
                                        'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'],
                            value = 'Enero',
                            layout = items_layout)

file_label = widget.HTML(value = f"<b style='color:white; font-family:tahoma; font-size:18px; border-radius: 3px 3px 3px 3px'>Nombre del archivo</b>")
file_output = widget.Text(value ='Reporte',                          
                          disable = False)

out = widget.Output(layout=box_layout)

# Salary Filter Button
calculate1 = widget.Button(description = 'Salario',
                          layout = items_layout,
                          style = button_style)                              
def on_calculate1_clicked(_):    
    with out:
        clear_output(wait=True)
        month_number = data_processing.switcher_months[month_menu.value]
        show_salary_on_date(year_menu.value, month_number)
calculate1.on_click(on_calculate1_clicked)

# Sub-Salary Filter Button
calculate2 = widget.Button(description='Sub-Salario',
                           layout=items_layout,
                           style = button_style)
def on_calculate2_clicked(_):    
    with out:
        clear_output(wait=True)
        month_number = data_processing.switcher_months[month_menu.value]
        show_worker_status_on_date(year_menu.value,month_number)
calculate2.on_click(on_calculate2_clicked)

# Work Filter Button
calculate3 = widget.Button(description='Trabajo',
                          layout=items_layout,
                          style = button_style)
def on_calculate3_clicked(_):    
    with out:
        clear_output(wait=True)
        month_number = data_processing.switcher_months[month_menu.value]        
        show_jobs_on_date(year_menu.value,month_number)     
calculate3.on_click(on_calculate3_clicked)

# Visualization Button
graph = widget.Button(description='Graficar',
                     layout=items_layout,
                     style = button_style)
def on_graph_clicked(_):
    with out:
        clear_output(wait=True)
        month = data_processing.switcher_months[month_menu.value]
        df_total_salary = pd.DataFrame(data_processing.total_salary_per_worker_date(year_menu.value,month))        
        # sorting the data
        df_total_salary.sort_values(by=['Salario Total'], inplace=True, ascending=True, ignore_index=True)
        display(data_graphs.plot_data(df_total_salary))         
graph.on_click(on_graph_clicked)

# Save Button
save = widget.Button(description='Guardar',
                     layout=items_layout,
                     style = button_style)
def on_save_clicked(_):
    with out:
        clear_output(wait=True)
        box = widget.Box([file_output, file_create], layout=box_layout)
        display(box)            
save.on_click(on_save_clicked)

# Create Button
file_create = widget.Button(description='Crear',
                            layout=items_layout,
                            style = button_style)
def on_create_clicked(_):
    with out:
        month = data_processing.switcher_months[month_menu.value]
        df_total_salary = pd.DataFrame(data_processing.total_salary_per_worker_date(year_menu.value,month))      
        df_worker_status = pd.DataFrame(data_processing.status_per_worker_date(year_menu.value, month))       
        t_worked = data_processing.job_done_per_worker_date(year_menu.value,month)
        df_worked = 0
        worker_list = []
        df_worked_list = []        
        for e in t_worked:
            worker_list.append(str(e[0]) + " " + str(e[1]) + " " + str(e[2]))
            df_worked = pd.DataFrame(e[3])    
            df_worked = df_worked.dropna(axis=0)
            df_worked_list.append(df_worked)        
        if not df_total_salary.empty and not df_worker_status.empty and not df_worked.empty :                        
            data_report.build_template(df_total_salary, df_worker_status, worker_list, df_worked_list, file_output.value)       
file_create.on_click(on_create_clicked)

# Error Button
errors = widget.Button(description='Errores',
                       layout=items_layout,
                       style = button_style)
def on_errors_clicked(_):
    with out:     
        clear_output(wait=True)
        show_model_error()        
errors.on_click(on_errors_clicked)

box1 = widget.VBox([year_menu, month_menu, calculate1, calculate2, calculate3, graph, errors, save], layout=box_layout2)
box2 = widget.HBox([out], layout=box_layout)

light_cols = ["Salario"]

def highlight_max(s):    
    if s.name in light_cols:
        is_max = s == s.max()
        return ['background-color: green' if v else '' for v in is_max]
    return [''] * len(s)

def highlight_min(s):        
    if s.name in light_cols:
        is_min = s == s.min()
        is_max = s == s.max()        
        mods = []
        for v in s:
            print(v)
            if v in is_min:
                mods.append('background-color: yellow')
            elif v in is_max:
                mods.append('background-color: green')
            else:
                mods.append('')       
        print(mods)
        return mods        
    return [''] * len(s)

def load_data():
    data_processing.traverse_dir()
    data_processing.asociate_status_with_jobs()
    
def show_salary_on_date(year, month):
    df_total_salary = pd.DataFrame(data_processing.total_salary_per_worker_date(year,month))
    # sorting the data
    df_total_salary.sort_values(by=['Salario Total'], inplace=True, ascending=False, ignore_index=True)
    df_total_salary = (df_total_salary.style                                           
                                            .set_table_styles(style_dataframe)
                                            .set_caption(str(year) + "/" + str(month))
                                            .format({'Salario Total':'$ {:.2f}'}))    
    display(df_total_salary)    
    
def show_worker_status():
    df_workers = pd.DataFrame(data_processing.status)
    df_workers = df_workers.drop(['Trabajado'], axis=1)
    df_workers = (df_workers.style
                                  .set_table_styles(style_dataframe)
                                  .set_caption("CONTROL DEL TRABAJO REALIZADO MENSUAL")
                                  .format({'Salario Escala':'$ {:d}', 'Salario':'$ {:.2f}'}))
    display(df_workers)
    
def show_worker_status_on_date(year, month):
    df_workers = pd.DataFrame(data_processing.status_per_worker_date(year, month))
    # sorting the data
    df_workers.sort_values(by=['C.I'], inplace=True, ascending=False, ignore_index=True)
    df_workers = (df_workers.style                                  
                                  .set_table_styles(style_dataframe)       
                                  .set_caption("CONTROL DEL TRABAJO REALIZADO MENSUAL")
                                  .format({'Salario Escala':'$ {:d}',
                                           'Salario':'$ {:.2f}'}))
    display(df_workers)

def show_jobs_on_date(year,month):
    t_worked = data_processing.job_done_per_worker_date(year,month)
    for e in t_worked:
        df_worked = pd.DataFrame(e[3])    
        df_worked = df_worked.drop([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
                                    11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
                                    21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], 
                                    axis=1)
        df_worked = df_worked.dropna(axis=0)
        df_worked = (df_worked.style                                    
                                    .set_table_styles(style_dataframe)
                                    .set_caption("Nombre: " + str(e[0]) + " C.I: " + str(e[1]) + " R.H: " + str(e[2]))
                                    .format({'TIEMPO DE REALIZACIÓN  (hrs)':'{:.2f}',
                                             'Salario':'$ {:d}', 
                                             'Tarifa Salarial Horaria (hrs)':'$ {:.2f}',
                                             'Importe por Remoto':'$ {:.2f}',
                                             'Importe por Interrupción':'$ {:.2f}',
                                             'Importe por Simultaneidad':'$ {:.2f}',
                                             'Nocturnidad':'$ {:.2f}',
                                             'Descuento por Calidad':'$ {:.2f}',
                                             'Importe por Programa':'$ {:.2f}'}))          
        display(df_worked)
        
def show_model_error():
    df_errors = pd.DataFrame(data_processing.non_valid)
    df_errors = (df_errors.style  
                                .apply(lambda x: ["background: #fd5352" if v == "Error" else "" for v in x], axis=1)                                  
                                .set_table_styles(style_dataframe)       
                                .set_caption("ERRORES"))                                  
    display(df_errors)

load_data()
display(box1)
display(box2)